## 0. Installing Requirements

In [1]:
!pip install hazm
!pip install pandas

<br>

## 1. Loading the Data

In [2]:
import pandas as pd

df = pd.read_json('./crawler/news_newformat_745.json')
df = df.transpose()

#### 1.1. Displaying Some Sample Records of Data

In [3]:
display(df.head())
print(f'The dataset contains {len(df)} records.')

,title,lead,text,tags,category,datetime,url
3063320,افشای تصاویر مهاجمین مسلح حمله تروریستی مسکو,تصاویر ۵ نفر از مهاجمین مسلح حمله تروریستی مسک...,رسانه‌ها، تصویر ۵ نفر از دستکم ۱۰ مهاجم عملیلت...,"[عملیات تروریستی, جنگ اوکراین]","[سیاسی, بین الملل]",2024-03-22T20:57:48Z,https://www.khabarfoori.com/بخش-%D8%B3%DB%8C%D...
3063319,مرگ تلخ ۳ عضو یک خانواده در تصادف رانندگی/ کود...,مدیر شبکه بهداشت و درمان پارس آباد از مرگ تلخ ...,دکتر منصور مهمان نواز با بیان اینکه تمامی فو...,"[پارس آباد, تصادف جاده ای, تصادف رانندگی, کشته...","[حوادث, تصادف]",2024-03-22T20:13:30Z,https://www.khabarfoori.com/بخش-%D8%AD%D9%88%D...
3063318,ایران حمله تروریستی در روسیه را محکوم کرد,سخنگوی وزارت امور خارجه کشورمان حمله تروریستی ...,ناصر کنعانی ضمن ابراز تسلیت به دولت و ملت روسی...,"[روسیه, سخنگوی وزارت امور خارجه, حادثه تروریستی]","[سیاسی, سیاست خارجی]",2024-03-22T20:07:36Z,https://www.khabarfoori.com/بخش-%D8%B3%DB%8C%D...
3063317,"پای چه‌کسانی در ""فاجعه کروکوس"" در میان است؟ / ...",عملیات تروریستی در مسکو تنها ساعاتی پس از آن ر...,جالب اینکه ۱۸ اسفند ماه سفارت آمریکا در روسیه ...,"[عملیات تروریستی, جنگ اوکراین]","[سیاسی, بین الملل]",2024-03-22T20:07:11Z,https://www.khabarfoori.com/بخش-%D8%B3%DB%8C%D...
3063316,رعنا آزادی ور عزادار شد,رعنا آزادی ور، بازیگر سینما و تلویزیون، چند سا...,رعنا آزادی ور متولد فروردین ماه سال 1362 می ب...,"[سوگ پدر, رعنا آزادی ور]","[فرهنگی, چهره ها]",2024-03-22T20:04:14Z,https://www.khabarfoori.com/بخش-%D9%81%D8%B1%D...


The dataset contains 744 records.


#### 1.2. Number of Data Records in Each Major Category

In [4]:
cats = df['category']
major_cats = cats.apply(lambda x: x[0])
display(major_cats.value_counts())

category
سیاسی        186
اقتصادی      142
ورزشی         88
فرهنگی        75
اجتماعی       66
حوادث         47
سرگرمی        46
استان‌ها      42
دانش          28
سبک زندگی     24
Name: count, dtype: int64

<br>

## 2. Data Preprocessing

#### 2.1. Combining All Textual Data for each Data Record

In [5]:
# Combining all textual data for each record in a single column
df['textual_data'] = df.apply(lambda row: ' '.join([
    row['title'],
    row['lead'],
    row['text'],
    *row['tags']
]), axis=1)

# Dropping unnecessay columns
df = df.drop(columns=[
    'title',
    'lead',
    'text',
    'tags',
    'datetime',
    'url',
])

# Retaining only the major category for each data record
df['category'] = df['category'].apply(lambda row: row[0])

display(df.head())

,category,textual_data
3063320,سیاسی,افشای تصاویر مهاجمین مسلح حمله تروریستی مسکو ت...
3063319,حوادث,مرگ تلخ ۳ عضو یک خانواده در تصادف رانندگی/ کود...
3063318,سیاسی,ایران حمله تروریستی در روسیه را محکوم کرد سخنگ...
3063317,سیاسی,"پای چه‌کسانی در ""فاجعه کروکوس"" در میان است؟ / ..."
3063316,فرهنگی,رعنا آزادی ور عزادار شد رعنا آزادی ور، بازیگر ...


#### 2.2. Text Normalization

In [6]:
from hazm import Normalizer


normalizer = Normalizer()
df['textual_data'] = df['textual_data'].apply(
    lambda row: normalizer.normalize(row))
display(df.head())

,category,textual_data
3063320,سیاسی,افشای تصاویر مهاجمین مسلح حمله تروریستی مسکو ت...
3063319,حوادث,مرگ تلخ ۳ عضو یک خانواده در تصادف رانندگی / کو...
3063318,سیاسی,ایران حمله تروریستی در روسیه را محکوم کرد سخنگ...
3063317,سیاسی,پای چه‌کسانی در «فاجعه کروکوس» در میان است؟ / ...
3063316,فرهنگی,رعنا آزادی ور عزادار شد رعنا آزادی ور، بازیگر ...


#### 2.3. Word Tokenization

In [7]:
from hazm import WordTokenizer


tokenizer = WordTokenizer()
df['textual_data'] = df['textual_data'].apply(
    lambda row: tokenizer.tokenize(row))
display(df.head())

,category,textual_data
3063320,سیاسی,"[افشای, تصاویر, مهاجمین, مسلح, حمله, تروریستی,..."
3063319,حوادث,"[مرگ, تلخ, ۳, عضو, یک, خانواده, در, تصادف, ران..."
3063318,سیاسی,"[ایران, حمله, تروریستی, در, روسیه, را, محکوم, ..."
3063317,سیاسی,"[پای, چه‌کسانی, در, «, فاجعه, کروکوس, », در, م..."
3063316,فرهنگی,"[رعنا, آزادی, ور, عزادار, شد, رعنا, آزادی, ور,..."


#### 2.4. Defining Stopwords

In [8]:
from hazm import stopwords_list

punctuations = [
    '.',
    '؟',
    '!',
    '،',
    ':',
    '؛',
    ')',
    '(',
    '»',
    '«',
    '«',
]
stopwords = set(stopwords_list() + punctuations)

#### 2.5. Stemming

In [9]:
from hazm import Stemmer


stemmer = Stemmer()
df['textual_data'] = df['textual_data'].apply(
    lambda row: [stemmer.stem(token) for token in row
                 if token not in stopwords])
display(df.head())

,category,textual_data
3063320,سیاسی,"[افشا, تصاویر, مهاجمین, مسلح, حمله, تروریست, م..."
3063319,حوادث,"[مرگ, تلخ, ۳, عضو, خانواده, تصادف, رانندگ, /, ..."
3063318,سیاسی,"[ایر, حمله, تروریست, روسیه, محکو, سخنگو, وزار,..."
3063317,سیاسی,"[پا, چه‌کسان, فاجعه, کروکوس, /, ناتو, اسرائیل,..."
3063316,فرهنگی,"[رعنا, آزاد, ور, عزادار, رعنا, آزاد, ور, بازیگ..."


#### 2.6. Lemmatization

In [10]:
from hazm import Lemmatizer


lemmatizer = Lemmatizer()
df['textual_data'] = df['textual_data'].apply(
    lambda row: [lemmatizer.lemmatize(token) for token in row
                 if token not in stopwords])
display(df.head())

,category,textual_data
3063320,سیاسی,"[افشا, تصاویر, مهاجمین, مسلح, حمله, تروریست, م..."
3063319,حوادث,"[مرگ, تلخ, ۳, عضو, خانواده, تصادف, رانندگ, /, ..."
3063318,سیاسی,"[ایر, حمله, تروریست, روسیه, محکو, سخنگو, وزار,..."
3063317,سیاسی,"[پا, چه‌کسان, فاجعه, کروکوس, /, ناتو, اسرائیل,..."
3063316,فرهنگی,"[رعنا, آزاد, ور, عزادار, رعنا, آزاد, ور, بازیگ..."


#### 2.7. Couting Occurences of each Token in each Category

In [11]:
import heapq
from operator import itemgetter


counts = {}
for _, row in df.iterrows():
    cat = row['category']
    for token in row['textual_data']:
        cat_dict = counts.setdefault(cat, {})
        token_cnt = cat_dict.setdefault(token, 0)
        cat_dict[token] = token_cnt + 1

for cat in counts.keys():
    print(f'Category = {cat}')
    tops = dict(heapq.nlargest(5, counts[cat].items(), key=itemgetter(1)))
    tops = dict(sorted(tops.items(), key=lambda x:x[1], reverse=True))
    print(tops, end='\n\n')

Category = سیاسی
{'کشور': 522, 'سال': 504, 'ایر': 430, 'مرد': 423, 'انتخاب': 367}

Category = حوادث
{'سال': 70, 'نوروز': 69, 'پلیس': 69, 'تصادف': 60, 'تهر': 56}

Category = فرهنگی
{'سال': 180, 'نوروز': 80, 'ایر': 70, 'سینما': 66, 'برنامه': 64}

Category = دانش
{'#هست': 50, 'سال': 50, 'کار': 49, 'استفاده': 43, 'آب': 42}

Category = سرگرمی
{'کرد#کن': 211, 'داد#ده': 210, 'کار': 204, 'امروز': 204, 'سال': 195}

Category = ورزشی
{'ت': 267, 'فوتبال': 184, 'ایر': 182, 'مل': 173, 'سال': 164}

Category = اقتصادی
{'سال': 470, 'درصد': 346, 'کار': 341, 'کارگر': 325, 'قیم': 307}

Category = اجتماعی
{'کشور': 189, 'سال': 146, '#هست': 136, 'جنوب': 103, 'گیاه': 102}

Category = استان‌ها
{'شرک': 80, 'مرد': 69, 'اس': 66, 'سال': 54, 'خوزس': 53}

Category = سبک زندگی
{'غذا': 70, 'مصرف': 69, '#هست': 61, 'مواد': 53, 'بدن': 48}

